This notebook generates a prediction for a user for a new book by averaging the ratings given by similar users for that book

In [1]:
import pandas as pd
test=pd.read_csv('test.csv')
user_cluster=pd.read_csv('user_cluster_final.csv')
train=pd.read_csv('latest_data/train.csv')

In [2]:
user_cluster.set_index('user_id',inplace=True)

In [3]:
user_cluster_test=test.merge(user_cluster,how='left',on='user_id')

In [16]:
book_cluster=user_cluster_test[['book_id','cluster']].drop_duplicates()

In [17]:
book_cluster=book_cluster.reset_index()

In [13]:
#since this is very computation heavy, we use swifter to make run the "apply" method more efficiently on pandas
import swifter 

In [18]:
def rating_for_book(book_id,cluster_number,index_):
    print(index_)
    similar_users=user_cluster[user_cluster['cluster']==cluster_number].index.to_list()
    
    train_temp=train[train['user_id'].isin(similar_users)]
    train_temp=train_temp[train_temp['book_id']==book_id]

    if len(train_temp)!=0:
        return train_temp['rating'].mean()
    else:
        return "Could not find"

In [19]:
book_cluster['predicted_rating']=book_cluster.swifter.apply(lambda x:rating_for_book(x['book_id'],x['cluster'],x['index']),axis=1)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [20]:
book_cluster.to_csv('book_cluster.csv')

In [21]:
book_cluster

,index,book_id,cluster,predicted_rating
0,0,14781501,2,3.430397
1,1,32073180,2,3.119048
2,2,438247,2,3.601594
3,3,162085,2,3.511613
4,4,32682,2,3.954136
...,...,...,...,...
20373,49985,13420212,2,3.625
20374,49989,6498354,2,3.857143
20375,49992,1154163,2,3.222222
20376,49995,700953,2,2.857143


In [24]:
final_output=user_cluster_test.merge(book_cluster,how='inner',on=['cluster','book_id'])

In [34]:
knn_result=final_output[['user_id','book_id','rating','predicted_rating']].drop_duplicates()

In [ ]:
knn_result.to_csv('kmeans_prediction.csv')

In [35]:
#It is possible that for some users, their cluster members may also have not read a book. These users have "Could not find" rating for that book.
filtered=knn_result[knn_result['predicted_rating']!='Could not find']
y_actual=filtered['rating']
y_predicted=filtered['predicted_rating']

In [ ]:
from sklearn.metrics import mean_squared_error

from math import sqrt

rmse = sqrt(mean_squared_error(y_actual, y_predicted))

print(rmse)